Target:

    1. Reach target accuracy of 99.4% with less than 20000 parameters and 20 epochs

Result:

    Best test accuracy: 99.49%
    Parameters: 18914

Analysis:

    1. 99.4% reaced at 11th epoch and stabilised later on

# Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

!pip install torchsummary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Select GPU or CPU in devices

In [2]:
# Device
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print("Device Selected:", device)

Device Selected: cuda


## Data Transformations

We need to augment the data to better represent the trainig set and define what the model sees during training. 

Some of the transformations which come pre-built with PyTorch

1. Resize
2. Scale
3. CenterCrop
4. Pad
5. Lambda
6. RandomApply

## Dataset and Creating Train/Test Split

In [8]:
batch_size = 128
torch.manual_seed(1)

kwargs = {'num_workers': 4, 'pin_memory': True} if device != 'cpu' else {}

train_loader = torch.utils.data.DataLoader(
                datasets.MNIST('../data', train=True, download=True,
                               transform=transforms.Compose([
                                  transforms.RandomRotation((-8.0, 8.0), center=None, fill=None),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                              ])),
                batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
                datasets.MNIST('../data', train=False, transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                              ])),
                batch_size=batch_size, shuffle=False, **kwargs)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Model definition

In [13]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Sequential(nn.Conv2d(1, 8, 3, padding=1, bias=False),       # In - 28x28x1, Out - 28x28x6, RF - 3x3
                                nn.ReLU(),
                                nn.BatchNorm2d(8),
                                nn.Conv2d(8, 16, 3, padding=1, bias=False),      # In - 28x28x6, Out - 28x28x12, RF - 5x5
                                nn.ReLU()
                              )

    self.transition1 = nn.Sequential(nn.MaxPool2d(2, 2))                        # In - 28x28x12, Out - 14x14x12, RF - 6x6

    self.conv2 = nn.Sequential(nn.Dropout(0.15),
                                nn.BatchNorm2d(16),
                                nn.Conv2d(16, 16, 3, padding=1, bias=False),     # In - 14x14x12, Out - 14x14x16, RF - 10x10
                                nn.ReLU(),
                                nn.BatchNorm2d(16),
                                nn.Conv2d(16, 32, 3, padding=1, bias=False),    # In - 14x14x16, Out - 14x14x16, RF - 14x14
                                nn.ReLU()
                              )

    self.transition2 = nn.Sequential(nn.MaxPool2d(2, 2))                        # In - 14x14x16, Out - 7x7x16, RF - 16x16

    self.conv3 = nn.Sequential(nn.Dropout(0.2),
                                nn.BatchNorm2d(32),
                                nn.Conv2d(32, 32, 3, padding=1, bias=False),    # In - 7x7x16, Out - 7x7x16, RF - 24x24
                                nn.ReLU()
                              )

    self.final_layers = nn.Sequential(nn.AvgPool2d(7, 7),
                                      nn.BatchNorm2d(32),
                                      nn.Conv2d(32, 32, 1, bias=False),         # In - 7x7x16, Out - 7x7x32, RF - 32x32
                                      nn.ReLU(),
                                      nn.Conv2d(32, 10, 1),                     # In - 7x7x32, Out - 7x7x10, RF - 32x32
                                      nn.Flatten(),
                                      nn.LogSoftmax()
                                      )

  def forward(self, x):
      x = self.conv1(x)
      x = self.transition1(x)
      x = self.conv2(x)
      x = self.transition2(x)
      x = self.conv3(x)
      x = self.final_layers(x)
      return x

# Model Summary

In [14]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4           [-1, 16, 28, 28]           1,152
              ReLU-5           [-1, 16, 28, 28]               0
         MaxPool2d-6           [-1, 16, 14, 14]               0
           Dropout-7           [-1, 16, 14, 14]               0
       BatchNorm2d-8           [-1, 16, 14, 14]              32
            Conv2d-9           [-1, 16, 14, 14]           2,304
             ReLU-10           [-1, 16, 14, 14]               0
      BatchNorm2d-11           [-1, 16, 14, 14]              32
           Conv2d-12           [-1, 32, 14, 14]           4,608
             ReLU-13           [-1, 32, 14, 14]               0
        MaxPool2d-14             [-1, 3

## Train and test functions 

In [15]:

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Train set: Average loss={loss.item()} batch_id={batch_idx}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return test_loss

# Defining optimizer + Training the model

In [16]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.4, verbose=True)

for epoch in range(0, 20):
    print("Epoch {}: ".format(epoch + 1))
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)
    scheduler.step(test_loss)

Epoch 1: 


Train set: Average loss=0.11322592943906784 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.48it/s]



Test set: Average loss: 0.0721, Accuracy: 9780/10000 (97.80%)

Epoch 2: 


Train set: Average loss=0.029797203838825226 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.44it/s]



Test set: Average loss: 0.0446, Accuracy: 9860/10000 (98.60%)

Epoch 3: 


...


Test set: Average loss: 0.0165, Accuracy: 9943/10000 (99.43%)

Epoch 19: 


Train set: Average loss=0.018927374854683876 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.70it/s]



Test set: Average loss: 0.0154, Accuracy: 9949/10000 (99.49%)

Epoch 20: 


Train set: Average loss=0.002033132826909423 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.41it/s]



Test set: Average loss: 0.0175, Accuracy: 9940/10000 (99.40%)



## Target validation accuracy of 99.4% reached in 11^th epoch